In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
labels = {0: 'hate speech', 1: 'offensive language', 2: 'neither'}

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification

teacher_id = 'bert-base-uncased'

def predict_distill(text, model_path):
    tokenizer = BertTokenizer.from_pretrained(teacher_id)
    model = BertForSequenceClassification.from_pretrained(model_path).to(device)
    model.eval()
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits,dim=1).item()
    return prediction

In [4]:
text_to_predict = 'At least I am not a nigga'
prediction = predict_distill(text_to_predict, 'saved_models/even_layers/')
print(f'predicted class by even_layers: {labels[prediction]}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at saved_models/even_layers/ and are newly initialized: ['bert.encoder.layer.10.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.output.dense.bias', 'bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.attention.self.key.bias', 'bert.encoder.layer.10.attention.self.key.weight', 'bert.encoder.layer.10.attention.self.query.bias', 'bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.layer.10.attention.self.value.bias', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.10.intermediate.dense.bias', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.10.output.LayerNorm.bias', 'bert.encoder.layer.10.output.LayerNorm.weight', 'bert.encoder.layer.10.output.dense.bias', 'bert.encoder.layer.10.output.dense.weight', 'bert.encoder.layer.11.attention.o

predicted class by even_layers: offensive language


In [5]:
prediction = predict_distill(text_to_predict, 'saved_models/odd_layers/')
print(f'predicted class by even_layers: {prediction}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at saved_models/odd_layers/ and are newly initialized: ['bert.encoder.layer.10.attention.output.LayerNorm.bias', 'bert.encoder.layer.10.attention.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.output.dense.bias', 'bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.attention.self.key.bias', 'bert.encoder.layer.10.attention.self.key.weight', 'bert.encoder.layer.10.attention.self.query.bias', 'bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.layer.10.attention.self.value.bias', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.10.intermediate.dense.bias', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.10.output.LayerNorm.bias', 'bert.encoder.layer.10.output.LayerNorm.weight', 'bert.encoder.layer.10.output.dense.bias', 'bert.encoder.layer.10.output.dense.weight', 'bert.encoder.layer.11.attention.ou

predicted class by even_layers: 1


In [6]:
from peft import PeftModel
from transformers import BertTokenizer, BertForSequenceClassification

def predict_lora(text, model_path):
    tokenizer = BertTokenizer.from_pretrained(teacher_id)
    base_model = BertForSequenceClassification.from_pretrained(teacher_id, num_labels=3)
    model = PeftModel.from_pretrained(base_model, model_path).to(device)
    model.eval()
    
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return prediction

In [7]:
text_to_predict = 'gay woke, woke gay'
prediction = predict_lora(text_to_predict, 'saved_models/lora/')
print(f'predicted class: {labels[prediction]}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\neucl\Dev\NLP\pytorch-cuda-12.4\Lib\site-packages\peft\peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.bert.encoder.layer.6.attention.self.query.lora_A.default.weight', 'base_model.model.bert.encoder.layer.6.attention.self.query.lora_B.default.weight', 'base_model.model.bert.encoder.layer.6.attention.self.value.lora_A.default.weight', 'base_model.model.bert.encoder.layer.6.attention.self.value.lora_B.default.weight', 'base_model.model.bert.encoder.layer.7.attention.self.query.lora_A.default.weight', 'base_model.model.bert.encoder.layer.7.attention.self.query.lora_B.default.weight', 'base_model.model.bert.encoder.layer.7.attention.

predicted class: offensive language
